### Project 04: Create a GUI Notebook Program

Sarina Li, David Lin, Rijah Ismail, Ezekiel Jatto

Project 4 will adapt the procedural code we have been working on in INST326_SimpleGUI_Note_Form_IO.ipynb to create an OOP version of the program using three classes.

    A Notebook or MainWindow class
    A Form or TopWindow class
    A Note class

The MainWindow class is a subclass of Tkinter’s tk.Tk class and thus inherits its attributes and methods, while allowing us to customize the new window objects to our needs. These new window objects will represent “notebooks” or collections of notes, and allow us to work with those notes. (I have named it MainWindow because this class definition could be used to create any kind of main window in Tkinter. We are using it to represent notebooks in this application, but you might use it for other purposes in onther applications.)


The TopWindow class creates a new top window in Tkinter, which is essentially a form for entering the title, text, links, and tags for our note. When we submit the note, this “form” object has a method that creates the note’s metadata and a new Note object. That note object is appended to the list of all notes, which is an attribute of the notebook (MainWindow) class.
The Note class creates note objects that contain the  title, text, links, tags, and metadata for each note.



#### Complete your code in the cell below

The code cell below contains the imports you will need; the top level structure for the three classes to get you started; and the execution code at the bottom. 

In [11]:
# Imports
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
import tkinter.messagebox as messagebox
import datetime # David: Module for working with dates and times
from functools import partial # Module for passing arguments in display_note command


class MainWindow(tk.Tk): 
    def __init__(self): # Initialize the main window super().init() 
        super().__init__()
        # Initialize as a tkinter window
        self.geometry("1400x800")# David: Set the default window size
        self.title('Notebook') # Set the default window title
        self.notebook = [] # Initialize an attribute named 'notebook' as an empty list    
    
    def new(self, note_or_snippet): # For creating new notes or code snippets
        if note_or_snippet == "note":
            # Initialize instance of NoteForm
            note_form = NoteForm(self, self.notebook)
        elif note_or_snippet == "snippet":
            # Initialize instance of CodeSnippetForm
            snippet_form = CodeSnippetForm(self, self.notebook)
        return None

    def open_(self, note_or_snippet): # For opening previously saved notes or code snippets in new window 
        # Sarina: Open file path, accepts .txt, .json, .csv, and .xml
        filepath = filedialog.askopenfilename(initialdir = "/Users/baihanliang/INST326-0104",
                                     filetypes=[("Allowed Types", "*.txt *.json *.csv *.xml")])
        try: 
            file = open(filepath, "r")
        except FileNotFoundError:
            print("It seems you didn't enter a valid file. Please try again.")
            
        file_list = file.read()
        
        # Notes:
        if note_or_snippet == "note":
            # Open new window for file text to be displayed
            note_window = tk.Toplevel() 
            note_window.geometry("1000x600")
            note_window.title("Note")
            text_label = tk.Label(note_window, text = file_list, justify = tk.LEFT)
            text_label.grid(padx = 10, pady = 10, row = 0, column = 0)
            file.close()
        # Code snippets:
        elif note_or_snippet == "snippet":
            # Open new window for file text to be displayed
            snippet_window = tk.Toplevel() 
            snippet_window.geometry("1000x600")
            snippet_window.title("Code Snippet")
            text_label = tk.Label(snippet_window, text = file_list, justify = tk.LEFT)
            text_label.grid(padx = 10, pady = 10, row = 0, column = 0)
            
            # Sarina: Create frame for 'copy code' button
            frame = tk.Frame(snippet_window)
            frame.grid(row = 1, column = 0)
            
            # Sarina: Isolate text
            sort_text = file_list.strip().split("\n")
            
            # Sarina: Create 'copy code' button
            b1 = tk.Button(frame, text = 'copy code', command = partial(self.copy, sort_text[1]))
            b1.grid(padx = 10, pady = 10, row = 0, column = 0, sticky = 'w')
        
            file.close()
        return None

    def display(self, to_display, note_or_snippet): # For displaying created notes or code snippets in new window
        # Create new window
        window = tk.Toplevel() 
        window.geometry("400x300")
        
        if note_or_snippet == "note":
            window.title("Note")
            display_text = f'\n{to_display.note_title}\n{to_display.note_text}\n{to_display.note_link}\n{to_display.note_tags}\n{to_display.note_meta}\n'
        elif note_or_snippet == "snippet":
            window.title("Code Snippet")
            display_text = f'\n{to_display.snippet_title}\n{to_display.snippet_text}\n{to_display.snippet_tags}\n{to_display.snippet_meta}\n'
            
        text_label = tk.Label(window, text = display_text, justify = tk.LEFT)
        text_label.grid(padx = 10, pady = 10, row = 0, column = 0)

    def save(self, note_or_snippet): # For saving notes or code snippets to computer
        # Ask to save note or code snippet as any file type user wishes, default is .txt
        # Sarina: Accepts .txt, .json, .csv, .xml
        file = filedialog.asksaveasfile(initialdir = "/Users/baihanliang/INST326-0104",
                                      defaultextension = ".txt",
                                      filetypes=[("Allowed Types", "*.txt *.json *.csv *.xml")])
    
        # Create frame for notes/code snippets to be displayed in main window 
        frame = tk.Frame(main_window)
        frame.grid(padx = 10, row = 0, column = 1)
        
        if note_or_snippet == "note":
            # Iterate through each note in 'notebook', writing each one to the file
            for note in self.notebook:
                file.write(f'\n{note.note_title}\n{note.note_text}\n{note.note_link}\n{note.note_tags}\n{note.note_meta}\n')
                # Add button representing note to main window
                note_button = tk.Button(frame, text = note.note_title, command = partial(main_window.display, note, note_or_snippet))
                note_button.grid(padx = 10, pady= 10, row = self.notebook.index(note)+1, column = 1)
        elif note_or_snippet == "snippet":
            # Iterate through each code snippet in 'notebook', writing each one to the file
            for snippet in self.notebook:
                file.write(f'\n{snippet.snippet_title}\n{snippet.snippet_text}\n{snippet.snippet_program}\n{snippet.snippet_tags}\n{snippet.snippet_meta}\n')
                # Add button representing snippet to main window
                snippet_button = tk.Button(frame, text = snippet.snippet_title, command = partial(main_window.display, snippet, note_or_snippet))
                snippet_button.grid(padx = 10, pady= 10, row = self.notebook.index(snippet)+1, column = 1)
        file.close()
        return None
    
    def edit(self, note_or_snippet): # For editing previously created notes or code snippets
        # Sarina: Open file path, accepts .txt, .json, .csv, .xml
        filepath = filedialog.askopenfilename(initialdir = "/Users/baihanliang/INST326-0104",
                                     filetypes=[("Allowed Types", "*.txt *.json *.csv *.xml")])
        try: 
            file = open(filepath, "r")
            
        except FileNotFoundError:
            print("Please select a valid file.")
            
        file_text = file.read()
        
        # Creating top level window
        window = tk.Toplevel() 
        window.geometry("400x350")
        frame = tk.Frame(window)
        frame.pack(expand = True)
        
        # Splitting current note contents up for default text in entry spaces
        file_list = file_text.strip().split("\n")
        title = file_list[0]
        text = file_list[1]
        
        # Title label
        title_label = tk.Label(frame, bg = 'dark gray', text = 'Note Title:')
        title_label.grid(padx = 10, pady = 10, row = 1, column = 0, sticky = 'e')
    
        # Text label
        text_label = tk.Label(frame, bg='dark gray', text='Note Text:')
        text_label.grid(padx = 10, pady = 10, row = 2, column = 0, sticky = 'e')
        
        # Create new title entry field
        title_entry = tk.Entry(frame, width = 80)
        title_entry.grid(padx = 10, pady = 10, row = 1, column = 1, sticky = 'w')
        title_entry.insert(0, title) # Adds default title

        # Create new text field
        text_entry = tk.Text(frame, height = 10, width = 60)
        text_entry.grid(padx = 10, pady = 10, row = 2, column = 1)
        text_entry.insert('1.0', text) # Adds default text
        
        # Meta data
        date_and_time = datetime.datetime.now()
        timezone = date_and_time.astimezone().tzinfo
        meta = f'created {date_and_time} {timezone}'
        
        if note_or_snippet == "note": # For notes
            # Creating window
            window.title("Note")
            
            # Extracting link and tags
            link = file_list[2]
            tags = file_list[3]
            
             # Link label
            link_label = tk.Label(frame, bg = 'dark gray', text = 'Note Link:')
            link_label.grid(padx = 10, pady = 10, row = 3, column = 0, sticky = 'e')
    
            # Tag label
            tag_label = tk.Label(frame, bg = 'dark gray', text = 'Note Tags:')
            tag_label.grid(padx = 10, pady = 10, row = 4, column = 0, sticky = 'e')
            
            # Create new link text field
            link_entry = tk.Entry(frame, width = 80)
            link_entry.grid(padx = 10, pady = 10, row = 3, column = 1)
            link_entry.insert(0, link)

            # Create new tags text field
            tags_entry = tk.Entry(frame, width = 80)
            tags_entry.grid(padx = 10, pady = 10, row = 4, column = 1) 
            tags_entry.insert(0, tags)
            
            # Create 'save changes' button
            b1 = tk.Button(frame, text = 'save changes', command = partial(self.submit_note_changes, filepath, title_entry, text_entry, link_entry, tags_entry, meta))
            b1.grid(padx = 10, pady = 10, row = 6, column = 1, sticky = 'w')
            
        elif note_or_snippet == "snippet": # For code snippets
            # Titling window
            window.title("Code Snippet")
            
            # Create 'save changes' button
            b1 = tk.Button(frame, text = 'save changes', command = partial(self.submit_snippet_changes, filepath, title_entry, text_entry, link_entry, tags_entry, meta))
            b1.grid(padx = 10, pady = 10, row = 6, column = 1, sticky = 'w')
    
        file.close()
        
    def copy(self, text_to_copy): # Sarina: For copying code snippets to clipboard
        self.clipboard_clear()
        self.clipboard_append(text_to_copy)

    def submit_note_changes(self, note_file, title, text, link, tags, creation_time): # For submitting note changes and tracking edit history
        # Extracting new note attributes
        new_title = title.get()
        new_text = text.get('1.0', 'end').strip("\n")
        new_link = link.get()
        new_tags = tags.get()
        new_note = f'{new_title}\\n{new_text}\\n{new_link}\\n{new_tags}\\n{creation_time}'
            
        # Create a dictionary to store the new note information
        edit_dict = {
            "timestamp": creation_time,
            "title": new_title,
            "text": new_text,
            "link": new_link,
            "tags": new_tags
        }
        
        # Create new note object and append to 'notebook'
        new_note_object = MakeNote(edit_dict)
        self.notebook.append(new_note_object)
            
        # Open the note file and read the contents
        with open(note_file, "r") as note:
            note_contents = note.readlines()
                
            # Find the existing note in the notebook list
            for existing_note in self.notebook:
                if existing_note.note_title == new_title:
                    # Append the edit dictionary to the edit history list of the existing note
                    existing_note.edit_history.append(edit_dict)
                        
                    # Overwrite the note file with the new changes
                    with open(note_file, "w") as note:
                        for edit in existing_note.edit_history:
                            writeable = f'{edit_dict["timestamp"]}\n{edit_dict["title"]}\n{edit_dict["text"]}\n{edit_dict["link"]}\n{edit_dict["tags"]}\n'
                            note.write(writeable)
                            # David: Display pop up window for user to confirm changes
                            messagebox.showinfo("Confirm Changes", writeable)
                            break
                        note.close()
    
    def submit_snippet_changes(self, snippet_file, title, text, program, tags, creation_time): # For submitting code snippet changes and tracking edit history
        # Extracting code snippet attributes
        new_title = title.get()
        new_text = text.get('1.0', 'end').strip("\n")
        new_program = program.get()
        new_tags = tags.get()
        new_snippet = f'{new_title}\\n{new_text}\\n{new_program}\\n{new_tags}\\n{creation_time}'
        
        # Create a dictionary to store the new code snippet information
        edit_dict = {
            "timestamp": creation_time,
            "title": new_title,
            "text": new_text,
            "link": new_link,
            "tags": new_tags
        }
        
        # Create new code snippet object and append to 'notebook'
        new_snippet_object = MakeCodeSnippet(edit_dict)
        self.notebook.append(new_snippet_object)
            
        # Open the code snippet file and read the contents
        with open(note_file, "r") as note:
            snippet_contents = note.readlines()
                
        # Find the existing note in the notebook list
        for existing_snippet in self.notebook:
            if existing_snippet.snippet_title == new_title:
                # Append the edit dictionary to the edit history list of the existing code snippet
                existing_snippet.edit_history.append(edit_dict)
                        
                # Overwrite the code snippet file with the new changes
                with open(note_file, "w") as snippet:
                    for edit in existing_snippet.edit_history:
                        writeable = f'{edit_dict["timestamp"]}\n{edit_dict["title"]}\n{edit_dict["text"]}\n'
                        snippet.write(writeable)
                        # David: Display pop up window for user to confirm changes
                        messagebox.showinfo("Confirm Changes", writeable)
                        break
                    note.close()
                    
    def delete(self, note_or_snippet): # Rijah: For deleting notes and code snippets
            # Open file path
            filepath = filedialog.askopenfilename(initialdir="/Users/baihanliang/INST326-0104",
                                                   filetypes=[("Allowed Types", "*.txt *.json *.csv *.xml")])
            try: 
                file = open(filepath, "r")
            except FileNotFoundError:
                print("It seems you didn't enter a valid file. Please try again.")

            file_list = file.read()
            file.close()

            if note_or_snippet == "note":
                for note in self.notebook:
                    if note.note_title in file_list:
                        self.notebook.remove(note)
                        messagebox.showinfo("Note Deleted", f"Note '{note.note_title}' has been deleted.")
                        break
            elif note_or_snippet == "snippet":
                for snippet in self.notebook:
                    if snippet.snippet_title in file_list:
                        self.notebook.remove(snippet)
                        messagebox.showinfo("Code Snippet Deleted", f"Code Snippet '{snippet.snippet_title}' has been deleted.")
                        break
    
    def search_notes(self, keyword): # Rijah: For searching for notes
        results = []
        for item in self.notebook:
            if keyword.lower() in item.note_text.lower() or keyword.lower() in item.note_title.lower():
                results.append(item)
        if results:
            search_result_window = tk.Toplevel()
            search_result_window.title("Search Results")
            for result in results:
                result_label = tk.Label(search_result_window, text=f"Title: {result.note_title}, Text: {result.note_text}")
                result_label.pack()
        else:
            messagebox.showinfo("Search Results", "No matching notes found.")
    
            
class NoteForm(tk.Toplevel):
    def __init__(self, master, notebook): # Initialize the new object
        super().__init__(master) # Initialize it as a Toplevel window
        self.geometry("600x350")
        self.title("New Note")
        self.notebook = notebook
        self.frame = tk.Frame(self) # Frame that covers the entire window
        self.frame.pack(fill = tk.BOTH, expand = True)
        self.frame.config(bg = "light gray")
        self.form_title = tk.Entry(self.frame, width = 80)
        self.text = tk.Text(self.frame, height = 10, width = 60)
        self.link = tk.Entry(self.frame, width = 80)
        self.tags = tk.Entry(self.frame, width = 80)
        self.date_and_time = datetime.datetime.now()
        self.timezone = self.date_and_time.astimezone().tzinfo
        self.meta = f'created {self.date_and_time} {self.timezone}'
    
        # Title label
        title_label = tk.Label(self.frame, bg = 'light gray', text = 'Note Title:')
        title_label.grid(padx = 10, pady = 10, row = 1, column = 0, sticky = 'e')
    
        # Text label
        text_label = tk.Label(self.frame, bg='light gray', text='Note Text:')
        text_label.grid(padx = 10, pady = 10, row = 2, column = 0, sticky = 'e')
    
        # Link label
        link_label = tk.Label(self.frame, bg = 'light gray', text = 'Note Link:')
        link_label.grid(padx = 10, pady = 10, row = 3, column = 0, sticky = 'e')
    
        # Tag label
        tag_label = tk.Label(self.frame, bg = 'light gray', text = 'Note Tags:')
        tag_label.grid(padx = 10, pady = 10, row = 4, column = 0, sticky = 'e')

        # Create note title entry field
        noteform_title = self.form_title
        noteform_title.grid(padx = 10, pady = 10, row = 1, column = 1, sticky = 'w')
        noteform_title.insert(0, 'New note title') # Adds default title

        # Create note text field
        noteform_text = self.text
        noteform_text.grid(padx = 10, pady = 10, row = 2, column = 1)
        noteform_text.insert('1.0', "Enter text here") # Adds default text

        # Create link text field
        noteform_link = self.link
        noteform_link.grid(padx = 10, pady = 10, row = 3, column = 1)
        noteform_link.insert(0, "Enter a relevant link here")

        # Create tags text field
        noteform_tags = self.tags
        noteform_tags.grid(padx = 10, pady = 10, row = 4, column = 1) 
        noteform_tags.insert(0, "Enter any relevant tags, separated by #")

        # Create 'submit' button
        b1 = tk.Button(self.frame, text = 'submit', command = self.submit)
        b1.grid(padx = 10, pady = 10, row = 6, column = 1, sticky = 'w')

    
    def submit(self): # For storing created notes in 'notebook'
        # Creating note_dict, which contains user inputs for note attributes
        
        title = self.form_title.get()
        text = self.text.get('1.0', 'end').strip('\n')
        link = self.link.get()
        tags = self.tags.get()
        note_dict = {'title':title, 'text':text, 'link': link, 'tags': tags, 'meta': self.meta}
    
        # Create note object and append to 'notebook'
        new_note = MakeNote(note_dict)
        self.notebook.append(new_note)
    
        # Print note contents
        printable = f'{title}\n{text}\n{link}\n{tags}\n{self.meta}'
        
        # Sarina: Display pop up window for user to confirm submission
        messagebox.showinfo("Note Details", printable)

        
class MakeNote(): # For creating note objects
    def __init__(self, note_dict):
        self.note_title = note_dict.get("title")
        self.note_text = note_dict.get("text")
        self.note_link = note_dict.get("link")
        self.note_tags = note_dict.get("tags")
        self.note_meta = note_dict.get("meta")
        
        # Initialize the edit history as an empty list
        self.edit_history = []
    
        # Add the initial creation to the edit history
        self.edit_history.append({
            "timestamp": self.note_meta,
            "title": self.note_title,
            "text": self.note_text,
            "link": self.note_link,
            "tags": self.note_tags
        })

        
class CodeSnippetForm(tk.Toplevel):
    def __init__(self, master, notebook): # initialize the new object
        super().__init__(master)# Initialize it as a Toplevel window
        self.geometry("600x350")
        self.title("New Code Snippet")
        self.notebook = notebook
        self.frame = tk.Frame(self) # Frame that covers the entire window
        self.frame.pack(fill = tk.BOTH, expand = True)
        self.frame.config(bg = "light gray")
        self.snippet_title = tk.Entry(self.frame, width = 80)
        self.snippet_text = tk.Text(self.frame, height = 10, width = 60)
        self.snippet_program = tk.Entry(self.frame, width = 80)
        self.snippet_tags = tk.Entry(self.frame, width = 80)
        self.date_and_time = datetime.datetime.now()
        self.timezone = self.date_and_time.astimezone().tzinfo
        self.snippet_meta = f'created {self.date_and_time} {self.timezone}'
        
        # Title label
        title_label = tk.Label(self.frame, bg = 'light gray', text = 'Snippet Title:')
        title_label.grid(padx = 10, pady = 10, row = 0, column = 0, sticky = 'e')
        
        # Text label
        text_label = tk.Label(self.frame, bg='light gray', text='Code:')
        text_label.grid(padx = 10, pady = 10, row = 1, column = 0, sticky = 'e')
        
        # Link Label
        program_label = tk.Label(self.frame, bg = 'light gray', text = 'Programs:')
        program_label.grid(padx = 10, pady = 10, row = 2, column = 0, sticky = 'e')
        
        tags_label = tk.Label(self.frame, bg = 'light gray', text = "Tags:")
        tags_label.grid(padx = 10, pady = 10, row = 3, column = 0, sticky = 'e')
    
        
        # Create note title entry field
        noteform_title = self.snippet_title
        noteform_title.grid(padx = 10, pady = 10, row = 0, column = 1, sticky = 'w')
        noteform_title.insert(0, 'New code snippet title') # Adds default title

        # Create note text field
        form_text = self.snippet_text
        form_text.grid(padx = 10, pady = 10, row = 1, column = 1)
        form_text.insert('1.0', "Enter code here") # Adds default text
        
        # Create link text field 
        snippet_program_text = self.snippet_program 
        snippet_program_text.grid(padx = 10, pady = 10, row = 2, column = 1)
        snippet_program_text.insert(0, 'Insert which programs the snippet can be used for here')
        
        # Create tag text field
        snippet_tags_text = self.snippet_tags
        snippet_tags_text.grid(padx = 10, pady = 10, row = 3, column = 1)
        snippet_tags_text.insert(0, 'Please insert Tags associated with the snippet here, remove text here if there are no tags') 

        # Create 'submit' button
        b1 = tk.Button(self.frame, text = 'submit', command = self.submit)
        b1.grid(padx = 10, pady = 10, row = 4, column = 1, sticky = 'w')
        
        
    def submit(self): # For storing created notes in 'notebook'
        # Creating note_dict, which contains user inputs for note attributes
        title = self.snippet_title.get()
        text = self.snippet_text.get('1.0', 'end').strip('\n')
        program = self.snippet_program.get()
        tags = self.snippet_tags.get()
        snippet_dict = {'title':title, 'text':text, 'program': program, 'tags': tags, 'meta': self.snippet_meta}
    
        # Create note object and append to 'notebook'
        new_snippet = MakeCodeSnippet(snippet_dict)
        self.notebook.append(new_snippet)
    
        # Print snippet contents
        printable = f'{title}\n{text}\n{program}\n{tags}\n{self.snippet_meta}'
        
        # Sarina: Display pop up window for user to confirm submission
        messagebox.showinfo("Snippet Details", printable)
        
        
class MakeCodeSnippet(): # For creating code snippet objects
    def __init__(self, snippet_dict):
        self.snippet_title = snippet_dict.get("title")
        self.snippet_text = snippet_dict.get("text")
        self.snippet_program = snippet_dict.get("program")
        self.snippet_tags = snippet_dict.get("tags")
        self.snippet_meta = snippet_dict.get("meta")
        
        # Initialize the edit history as an empty list
        self.edit_history = []
    
        # Add the initial creation to the edit history
        self.edit_history.append({
            "timestamp": self.snippet_meta,
            "title": self.snippet_title,
            "text": self.snippet_text,
            "link": self.snippet_program,
            "tags": self.snippet_tags
        })      
    
        
# Main execution
if __name__ == '__main__':
    
    # Create a notebook
    main_window = MainWindow() 

    # Label for placing buttons
    button_label = tk.Label(main_window)
    button_label.grid(padx = 10, pady = 10, row = 0, column = 0)

    # New note button
    new_note = tk.Button(button_label, text = 'new note', command = partial(main_window.new, "note"))
    new_note.grid(padx=10, pady=10, row=5, column=0)

    # Open note button
    open_note = tk.Button(button_label, text='open note', command = partial(main_window.open_, "note"))
    open_note.grid(padx=10, pady=10, row=6, column=0)

    # Save note button
    save_note = tk.Button(button_label, text='save note', command = partial(main_window.save, "note"))
    save_note.grid(padx=10, pady=10, row=7, column=0) 

    # Edit note button
    edit_note = tk.Button(button_label, text='edit note', command = partial(main_window.edit, "note"))
    edit_note.grid(padx=10, pady=10, row=8, column=0)
    
    # New code snippet button
    new_note = tk.Button(button_label, text = 'new code snippet', command = partial(main_window.new, "snippet"))
    new_note.grid(padx=10, pady=10, row=5, column=1)

    # Open code snippet button
    open_note = tk.Button(button_label, text='open code snippet', command = partial(main_window.open_, "snippet"))
    open_note.grid(padx=10, pady=10, row=6, column=1)

    # Save code snippet button
    save_note = tk.Button(button_label, text='save code snippet', command = partial(main_window.save, "snippet"))
    save_note.grid(padx=10, pady=10, row=7, column=1) 

    # Edit code snippet button
    edit_note = tk.Button(button_label, text='edit code snippet', command = partial(main_window.edit, "snippet"))
    edit_note.grid(padx=10, pady=10, row=8, column=1)
    
    # Rijah: Search note button
    search_button = tk.Button(button_label, text='Search Notes', command=lambda: main_window.search_notes(search_entry.get()))
    search_button.grid(padx=10, pady=10, row=9, column=0)

    # Rijah: Collecting user input for search note
    search_entry = tk.Entry(button_label)
    search_entry.grid(padx=10, pady=10, row=9, column=1)  
    
    # Rijah: Delete note button
    delete_note = tk.Button(button_label, text='delete note', command=partial(main_window.delete, "note"))
    delete_note.grid(padx=10, pady=10, row=10, column=0)
    
    # Rijah: Delete code snippet button
    delete_snippet = tk.Button(button_label, text='delete code snippet', command=partial(main_window.delete, "snippet"))
    delete_snippet.grid(padx=10, pady=10, row=10, column=1)
    
    # Rijah: Configuring display of new, open, save, and edit buttons
    # Note: Button colors may not appear on Mac interfaces due to known issues with the Mac tcl (https://www.python.org/download/mac/tcltk/)
    new_note.config(font=("Arial", 12), bg="lightblue", fg="black")
    open_note.config(font=("Arial", 12), bg="lightgreen", fg="black")
    save_note.config(font=("Arial", 12), bg="lightcoral", fg="black")
    edit_note.config(font=("Arial", 12), bg="lightyellow", fg="black")
    
    # Rijah: Configuring main window display
    main_window.config(bg="lightgray")

main_window.mainloop()

## Print your three notes below

In [ ]:
# print your notes here

In [ ]:
for note in main_window.notebook:
    print("\n")
    print(note)